<a href="https://colab.research.google.com/github/LuisErnestoColchado/Learning-Deep-Pollution/blob/master/GAN_STL_Beijing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys

In [1]:
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 636.8MB 27kB/s 
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 1.0.1.post2 which is incompatible.
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


In [2]:
!pip3 install torchvision

     |████████████████████████████████| 676.9MB 20kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import datetime
print("Cuda is available in Pytorch: ",torch.cuda.is_available())

Cuda is available in Pytorch:  True


# Read Data


In [0]:
aqi_beijing_data = pd.read_csv("/content/drive/My Drive/masterThesis/GAN_SpatioTemporal/data/beijing_17_18_aq.csv")
meo_beijing_data = pd.read_csv("/content/drive/My Drive/masterThesis/GAN_SpatioTemporal/data/beijing_17_18_meo.csv")

# Preprocessing Data


## Air Quality Data

In [6]:
aqi_beijing_data.head()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin_aq,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin_aq,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin_aq,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin_aq,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin_aq,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0


In [8]:
aqi_beijing_data.describe()

,PM2.5,PM10,NO2,CO,O3,SO2
count,290621.000000,227747.000000,292359.000000,268197.000000,290589.000000,292462.000000
mean,58.785570,88.059259,45.792457,0.960677,55.692755,8.981003
std,66.113851,89.292875,32.065045,1.001856,53.821424,11.702594
min,2.000000,5.000000,1.000000,0.100000,1.000000,1.000000
25%,16.000000,37.000000,20.000000,0.400000,12.000000,2.000000
50%,39.000000,70.000000,39.000000,0.700000,45.000000,5.000000
75%,77.000000,113.000000,66.000000,1.200000,79.000000,11.000000
max,1004.000000,3000.000000,300.000000,15.000000,504.000000,307.000000


In [0]:
aqi_beijing_data["utc_time"] = aqi_beijing_data.index
used_times = []
for index in aqi_beijing_data.index:
    time = aqi_beijing_data.loc[index]["utc_time"]
    if time not in used_times :
        used_times.append(time)
    else : 
        aqi_beijing_data.drop([index], inplace=True)

In [13]:
aqi_beijing_data.describe()

,utc_time,PM2.5,PM10,NO2,CO,O3,SO2,time
count,311010.000000,290621.000000,227747.000000,292359.000000,268197.000000,290589.000000,292462.000000,311010.000000
mean,155504.500000,58.785570,88.059259,45.792457,0.960677,55.692755,8.981003,155504.500000
std,89780.997948,66.113851,89.292875,32.065045,1.001856,53.821424,11.702594,89780.997948
min,0.000000,2.000000,5.000000,1.000000,0.100000,1.000000,1.000000,0.000000
25%,77752.250000,16.000000,37.000000,20.000000,0.400000,12.000000,2.000000,77752.250000
50%,155504.500000,39.000000,70.000000,39.000000,0.700000,45.000000,5.000000,155504.500000
75%,233256.750000,77.000000,113.000000,66.000000,1.200000,79.000000,11.000000,233256.750000
max,311009.000000,1004.000000,3000.000000,300.000000,15.000000,504.000000,307.000000,311009.000000


## Meteorological Data

In [7]:
meo_beijing_data.head()

,station_id,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed,weather
0,shunyi_meo,116.615278,40.126667,2017-01-30 16:00:00,-1.7,1028.7,15,215.0,1.6,Sunny/clear
1,shunyi_meo,116.615278,40.126667,2017-01-30 17:00:00,-3.5,1028.4,24,16.0,1.0,Haze
2,shunyi_meo,116.615278,40.126667,2017-01-30 18:00:00,-3.7,1028.1,27,32.0,1.1,Haze
3,shunyi_meo,116.615278,40.126667,2017-01-30 19:00:00,-3.9,1027.6,27,21.0,0.9,Haze
4,shunyi_meo,116.615278,40.126667,2017-01-30 20:00:00,-4.1,1026.7,26,17.0,1.1,Haze


In [14]:
meo_beijing_data.describe()

,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed
count,158047.000000,158047.000000,158047.000000,158047.000000,158047.000000,157813.000000,157813.000000
mean,116.395672,40.047275,38.182775,1026.798200,354.312894,35487.471178,96.933881
std,0.395223,0.264798,5030.694816,5025.748455,17423.723518,184454.823456,9748.857634
min,115.692222,39.718611,-21.300000,940.000000,4.000000,0.000000,0.000000
25%,116.194167,39.847500,2.500000,994.200000,27.000000,78.000000,0.900000
50%,116.290556,39.973889,13.800000,1005.600000,48.000000,190.000000,1.500000
75%,116.626944,40.223333,23.200000,1016.900000,73.000000,280.000000,2.500000
max,117.117778,40.658889,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000


In [0]:
meo_beijing_data["utc_time"] = meo_beijing_data.index
used_times = []
for index in meo_beijing_data.index:
    time = meo_beijing_data.loc[index]["utc_time"]
    if time not in used_times :
        used_times.append(time)
    else : 
        meo_beijing_data.drop([index], inplace=True)

# GAN 

##Spatio-Temporal Learning

In [0]:
def EuclidianDistance(X,X_t):
  sum = 0
  for x in X:
    for x_t in X_t:
      sum += np.pow(x - x_t,2)
  return np.sqrt(sum)

In [0]:
def LostFunction(state_neigh, state_x):
  return (1/2) * np.pow(state_x-state_neigh,2) 

In [0]:
def functionCost(m,u,N_spa,N_tem,X,alpha,):
  C_spa = 0
  C_tem = 0
  L_spa = 0
  L_tem = 0
  Sum_spa = 0
  Sum_tem = 0
  for x in X:
    for n_spa in N_spa:
      C_spa = np.pow(sys.float_info.epsilon,-EuclidianDistance(x,n_spa))
      L_spa = LostFunction(n_spa,x)
      Sum_spa += C_spa * L_spa
    for n_tem in N_tem:
      C_tem = np.pow(sys.float_info.epsilon,-EuclidianDistance(x,n_tem))
      L_tem = LostFunction(n_tem,x)
      Sum_tem += C_tem * L_tem
  return ((1 / (m+u)) * Sum_spa) +  ((alpha/(m+u)) * Sum_tem) 

## Generator


In [0]:
class Generator(nn.Module):
  def __init__(self, input_size, output_size):
    
  
  def Forward(self, x):
    x = self.map1(x)
    x = 
  